In [1]:
import asyncio
import hypersync
from hypersync import BlockField, TransactionField, HypersyncClient
import polars as pl

In [2]:
def convert_hex_to_int(hex: str) -> int:
    """
    Converts hexadecimal values in a transaction dictionary to integers, skipping specific keys.

    Args:
    transaction (dict): A dictionary containing transaction data, where some values are hexadecimals.

    Returns:
    dict: A new dictionary with hexadecimals converted to integers, excluding specified keys.
    """
    # Only convert hex strings; leave other values as is
    if isinstance(hex, str) and hex.startswith("0x"):
        # Convert hex string to integer
        return int(hex, 16)
    else:
        return hex

In [3]:
async def firehose_tx_block_query():
    # Create hypersync client using the mainnet hypersync endpoint (default)
    client = hypersync.HypersyncClient()

    query = client.preset_query_blocks_and_transactions(19628000, 19629000)

    print("Running the query...")

    res = await client.send_req(query)

    print(
        f"Query returned {len(res.data.blocks)} blocks and {
            len(res.data.transactions)} transaction hashes"
    )

    return res.data.transactions, res.data.blocks


# Await the main() function and capture its returned value
all_txs, all_blocks = await firehose_tx_block_query()

Running the query...
Query returned 467 blocks and 74374 transaction hashes


In [4]:
all_txs[0]

Transaction { block_hash: Some("0x1df4377cf408bc7b50d3918cb49c1f3b0f668b49772a4466404edc72c3cf40a3"), block_number: 19628000, from_: Some("0xa3ab1bee0d2b87218971e2a8f4a73f29a57a6c15"), gas: Some("0x5208"), gas_price: Some("0x0a7a358200"), hash: Some("0xbb7c934c3bf41677af63dc67c715d6d190be96e756d470198562dc62789bfd56"), input: Some("0x"), nonce: Some("0x00"), to: Some("0x2f19e5c3c66c44e6405d4c200fe064ece9bc253a"), transaction_index: 0, value: Some("0x9adced66380bf000"), v: Some("0x01"), r: Some("0xd958a708cb975e97ad9a5972592b4e521ddc08c671df8b03016d3edb867c9ded"), s: Some("0x236f0391f86760ecac182010b07900d3f9b720c0f69afd5a12d4584205122f08"), max_priority_fee_per_gas: Some("0x0a7a358200"), max_fee_per_gas: Some("0x0a7a358200"), chain_id: Some("0x01"), cumulative_gas_used: Some("0x5208"), effective_gas_price: Some("0x0a7a358200"), gas_used: Some("0x5208"), contract_address: None, logs_bloom: Some("0x000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000

In [5]:
all_data = []
for tx in all_txs:
    tx_data = {
        "block_number": tx.block_number,
        "hash": tx.hash,
        "from_": tx.from_,
        "to": tx.to,
        "gas": convert_hex_to_int(tx.gas),
        "transaction_index": tx.transaction_index,
        "gas_price": convert_hex_to_int(tx.gas_price),
        "effective_gas_price": convert_hex_to_int(tx.effective_gas_price),
        "gas_used": convert_hex_to_int(tx.gas_used),
        "cumulative_gas_used": convert_hex_to_int(tx.cumulative_gas_used),
        "max_fee_per_gas": convert_hex_to_int(tx.max_fee_per_gas),
        "max_priority_fee_per_gas": convert_hex_to_int(tx.max_priority_fee_per_gas),
    }

    all_data.append(tx_data)

In [6]:
all_data_df = pl.DataFrame(all_data)

In [7]:
all_data_df.head(5)

block_number,hash,from_,to,gas,transaction_index,gas_price,effective_gas_price,gas_used,cumulative_gas_used,max_fee_per_gas,max_priority_fee_per_gas
i64,str,str,str,i64,i64,i64,i64,i64,i64,i64,i64
19628000,"""0xbb7c934c3bf4…","""0xa3ab1bee0d2b…","""0x2f19e5c3c66c…",21000,0,45000000000,45000000000,21000,21000,45000000000,45000000000
19628000,"""0xf5f61620a05a…","""0x78e7b9bd59b9…","""0x8207c1ffc5b6…",69024,1,22000000000,22000000000,39696,60696,null,null
19628000,"""0x8a132d193894…","""0xb4023ad7a46e…","""0x1f9840a85d5a…",68932,2,21284173218,21284173218,52418,113114,21284173218,21284173218
19628000,"""0x222ae08c0612…","""0x4f2f271e58e9…","""0xd37bbe5744d7…",44922,3,20000000000,20000000000,39836,152950,null,null
19628000,"""0x9a7c49692f21…","""0x077d360f11d2…","""0x19cbe5933034…",60000,4,19980000000,19980000000,21000,173950,null,null


In [8]:
# filtered tx query
async def filtered_tx_block_query():
    client = HypersyncClient()

    diff = 7200 * 2
    height = await client.get_height()

    query = hypersync.Query(
        from_block=height - diff,
        transactions=[
            hypersync.TransactionSelection(
                from_=["0x5050f69a9786f081509234f1a7f4684b5e5b76c9"]
            )
        ],
        field_selection=hypersync.FieldSelection(
            transaction=[el.value for el in TransactionField],
            block=[el.value for el in BlockField],
        ),
    )

    transactions = []
    blocks = []

    while True:
        res = await client.send_req(query)

        transactions += res.data.transactions
        blocks += res.data.blocks

        if res.archive_height < res.next_block:
            # quit if reached tip (can put a time.sleep here and continue)
            break

        query.from_block = res.next_block

        print("scanned up to block " + str(query.from_block))

    return transactions, blocks


# Await the main() function and capture its returned value
transactions, blocks = await filtered_tx_block_query()

In [9]:
data = []
for tx in transactions:
    tx_data = {
        "block_number": tx.block_number,
        "hash": tx.hash,
        "from_": tx.from_,
        "to": tx.to,
        "gas": convert_hex_to_int(tx.gas),
        "transaction_index": tx.transaction_index,
        "gas_price": convert_hex_to_int(tx.gas_price),
        "effective_gas_price": convert_hex_to_int(tx.effective_gas_price),
        "gas_used": convert_hex_to_int(tx.gas_used),
        "cumulative_gas_used": convert_hex_to_int(tx.cumulative_gas_used),
        "max_fee_per_gas": convert_hex_to_int(tx.max_fee_per_gas),
        "max_priority_fee_per_gas": convert_hex_to_int(tx.max_priority_fee_per_gas),
    }

    data.append(tx_data)

In [10]:
df = pl.DataFrame(data)

In [11]:
df.head(5)

block_number,hash,from_,to,gas,transaction_index,gas_price,effective_gas_price,gas_used,cumulative_gas_used,max_fee_per_gas,max_priority_fee_per_gas
i64,str,str,str,i64,i64,i64,i64,i64,i64,i64,i64
19614730,"""0xa80e5b0868bb…","""0x5050f69a9786…","""0xff0000000000…",21000,21,25634302908,25634302908,21000,822429,49147568278,2000000000
19614736,"""0x79224a55c0c4…","""0x5050f69a9786…","""0xff0000000000…",21000,56,21904177440,21904177440,21000,1719094,46282936838,2000000000
19614742,"""0xf40e357ceac7…","""0x5050f69a9786…","""0xff0000000000…",21000,45,24024458604,24024458604,21000,4148942,37602652280,2000000000
19614744,"""0x96d5370232f2…","""0x5050f69a9786…","""0xff0000000000…",21000,71,23692359114,23692359114,21000,5304387,45334028610,2000000000
19614750,"""0x373eaf9d4a89…","""0x5050f69a9786…","""0xff0000000000…",21000,14,23647749138,23647749138,21000,1568409,44382244956,2000000000
